In [2]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as offline
import pandas as pd 
import numpy as np
import plotly.io as pio
from scipy import stats 
offline.init_notebook_mode(connected=True)
from IPython.display import display
from math import ceil




In [3]:
arr=!echo $LOGDIR
FOLDER=arr[0]

In [4]:
#Place / at the end of path
FOLDER="/zfs/dicelab/DigitalPlant/results/May_27_16_57_49_node1672/"
# pd.set_option('display.max_columns', None)


In [5]:
#FOLDER="./"
df = pd.read_csv(FOLDER+'all.csv', na_filter= False)
df = df.sort_values(['deviceNum','Message ID'])

# devices = df.groupby('deviceNum')
# print(devices)
df['IMT'] =  np.where(df['Message ID']== 0, np.nan, df['Send Time']- df['Send Time'].shift(1))


df['Receive Time'] = pd.to_datetime(df['Receive Time'], unit='ms')
df['Send Time'] = pd.to_datetime(df['Send Time'], unit='ms')
df['Latency'] = (df['Receive Time'] - df['Send Time']).dt.total_seconds() * 1000
# df['Latency'] = (df['Latency'].dt.total_seconds() * 1000)
display(df)

Message ID               Send Time            Receive Time  Status  \
0               0 1970-03-05 01:51:11.811 1970-03-05 01:51:12.063       1   
1               1 1970-03-05 01:51:11.861 1970-03-05 01:51:12.073       1   
2               2 1970-03-05 01:51:11.942 1970-03-05 01:51:12.088       1   
3               3 1970-03-05 01:51:11.994 1970-03-05 01:51:12.106       1   
4               4 1970-03-05 01:51:12.044 1970-03-05 01:51:12.117       1   
...           ...                     ...                     ...     ...   
80995        2995 1970-03-05 01:53:45.667 1970-03-05 01:53:48.439       0   
80996        2996 1970-03-05 01:53:45.717 1970-03-05 01:53:48.439       0   
80997        2997 1970-03-05 01:53:45.767 1970-03-05 01:53:48.439       0   
80998        2998 1970-03-05 01:53:45.817 1970-03-05 01:53:48.439       0   
80999        2999 1970-03-05 01:53:45.867 1970-03-05 01:53:48.439       0   

       deviceNum   IMT  Latency  
0              0   NaN    252.0  
1              0  50.0    212.0  
2              0  81.0    146.0  
3              0  52.0    112.0  
4              0  50.0     73.0  
...          ...   ...      ...  
80995       1023  51.0   2772.0  
80996       1023  50.0   2722.0  
80997       1023  50.0   2672.0  
80998       1023  50.0   2622.0  
80999       1023  50.0   2572.0  

[3049728 rows x 7 columns]

In [ ]:
!cat {FOLDER}parameters

In [ ]:
#Count, number of messages - first message in every block



In [ ]:
df.groupby(['Status']).count()

In [ ]:
df['count'] = df.groupby('deviceNum')['deviceNum'].transform('count')
df['count'].describe()
num = df.groupby('deviceNum')['deviceNum'].size()[0]
print(num)

In [ ]:
#get rid of the connection message
PERCENT_TO_DROP = .05

num_messages = df.groupby('deviceNum')['deviceNum'].size()[0]
messages_to_drop = int(ceil(num_messages*PERCENT_TO_DROP))

print("Original dataframe contained {} messages, with each device sending {}\nDropping {}%, or {} messages from each side".format(len(df), num_messages, PERCENT_TO_DROP,messages_to_drop))
df.drop(df[df['Message ID'] >= (num_messages - messages_to_drop)].index, inplace=True)
df.drop(df[df['Message ID'] < messages_to_drop].index, inplace=True) #0 1 2 3 4


print()
# df.count()

In [ ]:
# fig = px.line(df, x="Message ID", y="Latency", color="deviceNum" ,title='Latency', render_mode='webgl')
# fig.update_traces(line=dict(width=1))
# fig.update_layout(showlegend=False)
# fig.show()
df['IMT'].describe()

In [ ]:
df['Latency'].describe()

In [ ]:
totalMessages = df.count()['count']
freq = df.groupby(['Latency']).count()['count']  # the count of each latency group
rel = freq / totalMessages # Divide that by the total messages to get the relative count
cumsum = rel.cumsum() #Cumulative to get the cumualive frequencies
fig_cfd = go.Figure()
fig_cfd.add_trace(go.Scatter(x=cumsum.index.values, y=cumsum,mode='lines',name='lines'))
fig_cfd.update_layout(
    title = "CFD of Latency (ms)",
    xaxis_title = "Latency (ms)",
    yaxis_title = "Probability"
)
fig_cfd.show()

In [ ]:
fig_ts = px.scatter(df, x="Receive Time", y="Latency", color="deviceNum", title='Time Received vs Latency (ms) Colored by Device', render_mode='webgl')
fig_ts.update_traces(line=dict(width=1))
fig_ts.update_layout(
    showlegend=False,
    yaxis_title="Latency (ms)")
fig_ts.show()

In [ ]:
hist = px.histogram(df, x="Receive Time", y="Latency", histfunc='avg', title="Histogram of Average latencies")
hist.show()

In [ ]:
by_status_fig = px.scatter(df, x="Receive Time", y="Latency", color="Status", title='Time Received vs Latency (ms) Colored by Status')
# fig_ts = px.line(df, x="Receive Time", y="Latency", color="deviceNum", title='Time Received vs Latency (ms) Colored by Status', render_mode='webgl')
# fig_ts.update_traces(line=dict(width=1))
by_status_fig.update_layout(
    showlegend=False,
    yaxis_title="Latency (ms)")
by_status_fig.show()

In [ ]:
boxfig = px.box(df, y="Latency")
boxfig.show()